In [6]:
import numpy as np
import random
import pickle
import asyncio
import threading
import time
from datetime import datetime
from collections import defaultdict

# Dicionários de apoio

# Dicionario de posições correpondentes do tabuleiro no array
dicio = {
    '1,1': 0, '1,2': 1, '1,3': 2, '1,4': 3, '1,5': 4,
    '2,1': 5, '2,2': 6, '2,3': 7, '2,4': 8, '2,5': 9,
    '3,1': 10, '3,2': 11, '3,3': 12, '3,4': 13, '3,5': 14,
    '4,1': 15, '4,2': 16, '4,3': 17, '4,4': 18, '4,5': 19,
    '5,1': 20, '5,2': 21, '5,3': 22, '5,4': 23, '5,5': 24
}

# Posições inversas para traduzir índices do array para coordenadas do tabuleiro
dicio_inverso = {v: k for k, v in dicio.items()}

# Posições na borda (as únicas que podem ser movidas de acordo com as regras do Quixo)
posicoes_borda = np.array([
    '1,1', '1,2', '1,3', '1,4', '1,5',
    '2,1', '2,5', '3,1', '3,5',
    '4,1', '4,5', '5,1', '5,2', '5,3', '5,4', '5,5'
])


# Indices correspondentes a cada do tabuleiro no array
dicioLinhas = {
    'Linha 1': [0, 1, 2, 3, 4], 'Linha 2': [5, 6, 7, 8, 9], 'Linha 3': [10, 11, 12, 13, 14],
    'Linha 4': [15, 16, 17, 18, 19], 'Linha 5': [20, 21, 22, 23, 24],
    'Coluna 1': [0, 5, 10, 15, 20], 'Coluna 2': [1, 6, 11, 16, 21],
    'Coluna 3': [2, 7, 12, 17, 22], 'Coluna 4': [3, 8, 13, 18, 23],
    'Coluna 5': [4, 9, 14, 19, 24], 'Diagonal 1': [0, 6, 12, 18, 24],
    'Diagonal 2': [4, 8, 12, 16, 20]
}

# Classe para armazenar o histórico de jogadas
class HistoricoJogadas:
    def __init__(self, max_jogadas=10):
        self.jogadas = []
        self.max_jogadas = max_jogadas

    def adicionar_jogada(self, jogador, origem, destino):
        #Marca hora em que a jogada ocorreu
        timestamp = datetime.now().strftime("%H:%M:%S")
        jogada = {
            "jogador": jogador,
            "origem": origem,
            "destino": destino,
            "timestamp": timestamp
        }
        self.jogadas.append(jogada)
        if len(self.jogadas) > self.max_jogadas:
            self.jogadas.pop(0)

    def obter_ultimas_jogadas(self, quantidade=None):
        if quantidade is None or quantidade > len(self.jogadas):
            return self.jogadas
        return self.jogadas[-quantidade:]

    def imprimir_historico(self):
        print("\n=== HISTÓRICO DE JOGADAS ===")
        for i, jogada in enumerate(self.jogadas, 1):
            print(f"{i}. {jogada['jogador']} moveu de {jogada['origem']} para {jogada['destino']} ({jogada['timestamp']})")
        print("===========================\n")

# Classe do jogo
class Jogo:
    def __init__(self, tabuleiro=None, jogador='🔴'):
        #Criação do tabuleiro
        self.tabuleiro = np.full(25, "⬜", dtype=str) if tabuleiro is None else np.array(tabuleiro, dtype=str)
        self.jogador_atual = jogador
        self.historico = HistoricoJogadas()
        self.ultima_jogada_agente = None

    def turno(self):
        return self.jogador_atual

    def codificar_estado(self):
        # Codifica o estado do jogo como uma string para usar como chave na tabela Q
        estado_encoded = ""
        for peca in self.tabuleiro:
            if peca == "⬜":
                estado_encoded += "0"
            elif peca == "🔴":
                estado_encoded += "1"
            elif peca == "❌":
                estado_encoded += "2"
        return estado_encoded + "_" + ("1" if self.jogador_atual == "🔴" else "2")

    # Verifica quem venceu
    def venceu(self):
        for nome, indices in dicioLinhas.items():
            valores = [self.tabuleiro[i] for i in indices]
            if all(v == "❌" for v in valores) or all(v == "🔴" for v in valores):
                return True
        return False

    # Verifica se empatou
    def empate(self):
        return not self.venceu() and "⬜" not in self.tabuleiro

    # Calcula a função de utilidade para saber o melhor estado a seguir
    def calcular_utilidade(self, jogador):
        if self.venceu():
            return 1 if self.jogador_atual == jogador else -1

        pontos = 0
        for nome, indices in dicioLinhas.items():
            valores = [self.tabuleiro[i] for i in indices]
            if jogador == "❌":
                pontos += valores.count("❌") * 0.1
                pontos -= valores.count("🔴") * 0.1
            else:
                pontos += valores.count("🔴") * 0.1
                pontos -= valores.count("❌") * 0.1

        return pontos

    # Gera todos os mov válidos possíveis para o jogador atual
    def jogos_validos(self):
        filhos = [] # Armazena todas as jogadas válidas possíveis a partir do estado atual
        for pos in posicoes_borda: # Itera pelas posições da borda
            i = dicio[pos] # Encontra indice correspondente a coordenada
            if self.tabuleiro[i] == "⬜" or self.tabuleiro[i] == self.jogador_atual: # Verifica se é uma peça válida de ser jogada
                for destino in self.movimentos_validos(pos): # Encontra destinos válidos para a posição
                    novo = self.jogar((pos, destino)) # Simula a jogada e verifica se ela é válida
                    if novo:
                        filhos.append((pos, destino))
        return filhos # Retorna jogadas válidas

    #Fazer a jogada
    def jogar(self, movimento):
        origem_str, destino_str = movimento
        origem_i, origem_j = map(int, origem_str.split(','))
        destino_i, destino_j = map(int, destino_str.split(','))

        if origem_str not in dicio or destino_str not in dicio:
            return None

        origem = dicio[origem_str]
        destino = dicio[destino_str]

        if origem_str not in posicoes_borda:
            return None

        if self.tabuleiro[origem] != "⬜" and self.tabuleiro[origem] != self.jogador_atual:
            return None

        novo_tabuleiro = np.array(self.tabuleiro)
        if origem_i == destino_i:  # linha
            linha = origem_i
            indices = [dicio[f"{linha},{j}"] for j in range(1, 6)]
            if destino_j > origem_j:
                for k in range(0, 4):
                    novo_tabuleiro[indices[k]] = novo_tabuleiro[indices[k + 1]]
                novo_tabuleiro[indices[4]] = self.jogador_atual
            else:
                for k in range(4, 0, -1):
                    novo_tabuleiro[indices[k]] = novo_tabuleiro[indices[k - 1]]
                novo_tabuleiro[indices[0]] = self.jogador_atual
        elif origem_j == destino_j:  # coluna
            coluna = origem_j
            indices = [dicio[f"{i},{coluna}"] for i in range(1, 6)]
            if destino_i > origem_i:
                for k in range(0, 4):
                    novo_tabuleiro[indices[k]] = novo_tabuleiro[indices[k + 1]]
                novo_tabuleiro[indices[4]] = self.jogador_atual
            else:
                for k in range(4, 0, -1):
                    novo_tabuleiro[indices[k]] = novo_tabuleiro[indices[k - 1]]
                novo_tabuleiro[indices[0]] = self.jogador_atual
        else:
            return None

        proximo_jogador = "🔴" if self.jogador_atual == "❌" else "❌"
        novo_jogo = Jogo(novo_tabuleiro, proximo_jogador)
        novo_jogo.historico = self.historico
        return novo_jogo

    def movimentos_validos(self, origem_str):
        origem_i, origem_j = map(int, origem_str.split(','))
        destinos = []

        if origem_i == 1:
            destinos.append(f"5,{origem_j}")
        elif origem_i == 5:
            destinos.append(f"1,{origem_j}")

        if origem_j == 1:
            destinos.append(f"{origem_i},5")
        elif origem_j == 5:
            destinos.append(f"{origem_i},1")

        return destinos

    #Imprime tabuleiro
    def imprimir(self):
        return print("| " + self.tabuleiro[0] + " | " + self.tabuleiro[1] + " | " + self.tabuleiro[2]+ " | " + self.tabuleiro[3] + " | " + self.tabuleiro[4] + " | " + "\n" +
        "| " + self.tabuleiro[5] + " | " + self.tabuleiro[6] + " | " + self.tabuleiro[7]+ " | " + self.tabuleiro[8] + " | " + self.tabuleiro[9] + " | " + "\n" +
        "| " + self.tabuleiro[10] + " | " + self.tabuleiro[11] + " | " + self.tabuleiro[12]+ " | " + self.tabuleiro[13] + " | " + self.tabuleiro[14] + " | " + "\n" +
        "| " + self.tabuleiro[15] + " | " + self.tabuleiro[16] + " | " + self.tabuleiro[17]+ " | " + self.tabuleiro[18] + " | " + self.tabuleiro[19] + " | " + "\n" +
        "| " + self.tabuleiro[20] + " | " + self.tabuleiro[21] + " | " + self.tabuleiro[22]+ " | " + self.tabuleiro[23] + " | " + self.tabuleiro[24] + " | " + "\n")

# Classe para agente Q-Learning
class QLearningAgent:
    def __init__(self, jogador="❌", alpha=0.1, gamma=0.9, epsilon=0.1, epsilon_decay=0.995, min_epsilon=0.01):
        self.jogador = jogador
        self.alpha = alpha  # Taxa de aprendizagem
        self.gamma = gamma  # Fator de desconto
        self.epsilon = epsilon  # Taxa de exploração
        self.epsilon_decay = epsilon_decay  # Decaimento da taxa de exploração
        self.min_epsilon = min_epsilon  # Taxa mínima de exploração
        self.q_table = defaultdict(lambda: defaultdict(float))  # Tabela Q: {estado: {ação: valor}}
        self.historico_estados = []  # Para aprendizagem temporal
        self.historico_acoes = []
        self.historico_recompensas = []

    def codificar_acao(self, acao):
        # Codifica uma ação (tupla de origem e destino) como string
        return f"{acao[0]}_{acao[1]}"

    def decodificar_acao(self, acao_str):
        # Decodifica uma string de ação para tupla
        origem, destino = acao_str.split('_')
        return (origem, destino)

    def escolher_acao(self, jogo, modo="treino"):
        # Escolhe uma ação usando epsilon-greedy ou modo de teste
        estado = jogo.codificar_estado()
        acoes_validas = jogo.jogos_validos()

        if not acoes_validas:
            return None

        # Modo de teste: sempre escolhe a melhor ação conhecida
        if modo == "teste":
            melhor_acao = None
            melhor_valor = float('-inf')

            for acao in acoes_validas:
                acao_str = self.codificar_acao(acao)
                valor = self.q_table[estado][acao_str]
                if valor > melhor_valor:
                    melhor_valor = valor
                    melhor_acao = acao

            return melhor_acao if melhor_acao else random.choice(acoes_validas)

        # Modo de treino: epsilon-greedy
        if random.random() < self.epsilon:
            # Exploração: ação aleatória
            return random.choice(acoes_validas)
        else:
            # Exploração: melhor ação conhecida
            melhor_acao = None
            melhor_valor = float('-inf')

            for acao in acoes_validas:
                acao_str = self.codificar_acao(acao)
                valor = self.q_table[estado][acao_str]
                if valor > melhor_valor:
                    melhor_valor = valor
                    melhor_acao = acao

            return melhor_acao if melhor_acao else random.choice(acoes_validas)

    def calcular_recompensa(self, jogo_anterior, jogo_atual, acao):
        # Calcula a recompensa para uma ação específica
        # Recompensa básica por vitória/derrota
        if jogo_atual.venceu():
            if jogo_anterior.jogador_atual == self.jogador:
                return 100  # Vitória
            else:
                return -100  # Derrota

        if jogo_atual.empate():
            return 0  # Empate

        # Recompensas intermediárias baseadas na posição estratégica
        recompensa = 0

        # Analisa vantagens posicionais
        for nome, indices in dicioLinhas.items():
            valores_anterior = [jogo_anterior.tabuleiro[i] for i in indices]
            valores_atual = [jogo_atual.tabuleiro[i] for i in indices]

            # Conta peças do jogador em cada linha/coluna/diagonal
            pecas_anteriores = valores_anterior.count(self.jogador)
            pecas_atuais = valores_atual.count(self.jogador)

            # Recompensa por formar sequências
            if pecas_atuais > pecas_anteriores:
                recompensa += (pecas_atuais ** 2) * 2  # Recompensa exponencial por sequências

            # Penaliza por permitir sequências do oponente
            oponente = "🔴" if self.jogador == "❌" else "❌"
            pecas_oponente_anterior = valores_anterior.count(oponente)
            pecas_oponente_atual = valores_atual.count(oponente)

            if pecas_oponente_atual > pecas_oponente_anterior:
                recompensa -= (pecas_oponente_atual ** 2) * 1.5

        return recompensa

    def atualizar_q_table(self, estado, acao, recompensa, proximo_estado):
        # Atualiza a tabela Q usando a equação do Q-Learning
        acao_str = self.codificar_acao(acao)

        # Q(s,a) = Q(s,a) + α[r + γ * max Q(s',a') - Q(s,a)]
        q_atual = self.q_table[estado][acao_str]

        # Encontra o valor máximo Q para o próximo estado
        max_q_proximo = 0
        if proximo_estado in self.q_table:
            max_q_proximo = max(self.q_table[proximo_estado].values()) if self.q_table[proximo_estado] else 0

        # Atualização da tabela Q
        novo_q = q_atual + self.alpha * (recompensa + self.gamma * max_q_proximo - q_atual)
        self.q_table[estado][acao_str] = novo_q

    def treinar_episodio(self, oponente_type="random"):
        # Treina um episódio completo contra um oponente
        jogo = Jogo(jogador=self.jogador)
        historico_experiencias = []  # (estado, ação, recompensa, próximo_estado)

        max_jogadas = 50  # Evita jogos infinitos
        contador = 0

        while not jogo.venceu() and not jogo.empate() and contador < max_jogadas:
            if jogo.turno() == self.jogador:
                # Turno do agente Q-Learning
                estado_atual = jogo.codificar_estado()
                acao = self.escolher_acao(jogo, modo="treino")

                if acao is None:
                    break

                jogo_anterior = jogo
                jogo = jogo.jogar(acao)

                if jogo is None:
                    break

                # Calcula recompensa e armazena experiência
                recompensa = self.calcular_recompensa(jogo_anterior, jogo, acao)
                proximo_estado = jogo.codificar_estado()

                historico_experiencias.append((estado_atual, acao, recompensa, proximo_estado))

            else:
                # Turno do oponente
                if oponente_type == "random":
                    acoes_validas = jogo.jogos_validos()
                    if acoes_validas:
                        acao = random.choice(acoes_validas)
                        jogo = jogo.jogar(acao)
                elif oponente_type == "minimax":
                    acao = melhor_jogada(jogo, profundidade=2)
                    if acao:
                        jogo = jogo.jogar(acao)

                if jogo is None:
                    break

            contador += 1

        # Atualiza a tabela Q com todas as experiências do episódio
        for estado, acao, recompensa, proximo_estado in historico_experiencias:
            self.atualizar_q_table(estado, acao, recompensa, proximo_estado)

        # Decai epsilon
        if self.epsilon > self.min_epsilon:
            self.epsilon *= self.epsilon_decay

        # Retorna resultado do jogo
        if jogo.venceu():
            return "vitoria" if contador > 0 and historico_experiencias else "derrota"
        else:
            return "empate"

    def salvar_modelo(self, nome_arquivo="q_learning_model.pkl"):
        # Salva o modelo Q-Learning
        dados = {
            'q_table': dict(self.q_table),
            'epsilon': self.epsilon,
            'jogador': self.jogador,
            'alpha': self.alpha,
            'gamma': self.gamma
        }
        with open(nome_arquivo, 'wb') as f:
            pickle.dump(dados, f)

    def carregar_modelo(self, nome_arquivo="q_learning_model.pkl"):
        # Carrega um modelo Q-Learning
        try:
            with open(nome_arquivo, 'rb') as f:
                dados = pickle.load(f)

            self.q_table = defaultdict(lambda: defaultdict(float), dados['q_table'])
            self.epsilon = dados['epsilon']
            self.jogador = dados['jogador']
            self.alpha = dados['alpha']
            self.gamma = dados['gamma']
            return True
        except FileNotFoundError:
            print(f"Arquivo {nome_arquivo} não encontrado.")
            return False

# Classe para treinamento e avaliação
class TreinadorQLearning:
    def __init__(self, agente):
        self.agente = agente
        self.metricas = {
            'vitorias': 0,
            'derrotas': 0,
            'empates': 0,
            'episodios': 0
        }
        self.historico_metricas = []

    def treinar(self, num_episodios=10000, oponente="random", salvar_a_cada=1000, nome_arquivo="q_learning_model.pkl"):
        # Treina o agente por um número específico de episódios
        print(f"🤖 Iniciando treinamento por {num_episodios} episódios contra oponente {oponente}")
        print(f"💾 Salvando modelo a cada {salvar_a_cada} episódios")

        start_time = time.time()

        for episodio in range(1, num_episodios + 1):
            resultado = self.agente.treinar_episodio(oponente_type=oponente)

            # Atualiza métricas
            if resultado == "vitoria":
                self.metricas['vitorias'] += 1
            elif resultado == "derrota":
                self.metricas['derrotas'] += 1
            else:
                self.metricas['empates'] += 1

            self.metricas['episodios'] = episodio

            # Relatório de progresso
            if episodio % 100 == 0:
                taxa_vitoria = (self.metricas['vitorias'] / episodio) * 100
                print(f"Episódio {episodio}/{num_episodios} - "
                      f"Vitórias: {taxa_vitoria:.1f}% - "
                      f"Epsilon: {self.agente.epsilon:.3f}")

            # Salva modelo periodicamente
            if episodio % salvar_a_cada == 0:
                self.agente.salvar_modelo(nome_arquivo)
                print(f"💾 Modelo salvo no episódio {episodio}")

                # Armazena métricas históricas
                self.historico_metricas.append({
                    'episodio': episodio,
                    'vitorias': self.metricas['vitorias'],
                    'derrotas': self.metricas['derrotas'],
                    'empates': self.metricas['empates'],
                    'taxa_vitoria': (self.metricas['vitorias'] / episodio) * 100,
                    'epsilon': self.agente.epsilon
                })

        # Salva modelo final
        self.agente.salvar_modelo(nome_arquivo)

        tempo_total = time.time() - start_time
        print(f"\n🏁 Treinamento concluído em {tempo_total:.2f} segundos")
        self.imprimir_relatorio_final()

    def imprimir_relatorio_final(self):
        # Imprime relatório final do treinamento com resultados
        total = self.metricas['episodios']
        print(f"\n=== RELATÓRIO FINAL DO TREINAMENTO ===")
        print(f"Total de episódios: {total}")
        print(f"Vitórias: {self.metricas['vitorias']} ({(self.metricas['vitorias']/total)*100:.1f}%)")
        print(f"Derrotas: {self.metricas['derrotas']} ({(self.metricas['derrotas']/total)*100:.1f}%)")
        print(f"Empates: {self.metricas['empates']} ({(self.metricas['empates']/total)*100:.1f}%)")
        print(f"Epsilon final: {self.agente.epsilon:.4f}")
        print(f"Tamanho da tabela Q: {len(self.agente.q_table)} estados")
        print("=====================================\n")

    def avaliar_contra_minimax(self, num_jogos=100):
        # Avalia o agente Q-Learning contra o Minimax
        print(f"🔄 Avaliando Q-Learning vs Minimax em {num_jogos} jogos...")

        vitorias_q = 0
        vitorias_minimax = 0
        empates = 0

        for jogo_idx in range(num_jogos):
            # Alterna quem começa
            if jogo_idx % 2 == 0:
                resultado = self._jogar_q_vs_minimax(q_comeca=True)
            else:
                resultado = self._jogar_q_vs_minimax(q_comeca=False)

            if resultado == "q_venceu":
                vitorias_q += 1
            elif resultado == "minimax_venceu":
                vitorias_minimax += 1
            else:
                empates += 1

            if (jogo_idx + 1) % 20 == 0:
                print(f"Progresso: {jogo_idx + 1}/{num_jogos} jogos")

        print(f"\n=== RESULTADO Q-LEARNING vs MINIMAX ===")
        print(f"Q-Learning: {vitorias_q} vitórias ({(vitorias_q/num_jogos)*100:.1f}%)")
        print(f"Minimax: {vitorias_minimax} vitórias ({(vitorias_minimax/num_jogos)*100:.1f}%)")
        print(f"Empates: {empates} ({(empates/num_jogos)*100:.1f}%)")
        print("======================================\n")

        return {
            'q_learning': vitorias_q,
            'minimax': vitorias_minimax,
            'empates': empates,
            'total': num_jogos
        }

    def _jogar_q_vs_minimax(self, q_comeca=True):
        # Joga uma partida entre Q-Learning e Minimax
        jogo = Jogo(jogador="🔴" if q_comeca else "❌")
        max_jogadas = 50
        contador = 0

        while not jogo.venceu() and not jogo.empate() and contador < max_jogadas:
            if (q_comeca and jogo.turno() == "🔴") or (not q_comeca and jogo.turno() == "❌"):
                # Turno do Q-Learning
                acao = self.agente.escolher_acao(jogo, modo="teste")
                if acao:
                    jogo = jogo.jogar(acao)
            else:
                # Turno do Minimax
                acao = melhor_jogada(jogo, profundidade=2)
                if acao:
                    jogo = jogo.jogar(acao)

            if jogo is None:
                break

            contador += 1

        if jogo and jogo.venceu():
            # Determina quem venceu baseado no último jogador
            ultimo_jogador = "❌" if jogo.turno() == "🔴" else "🔴"
            if (q_comeca and ultimo_jogador == "🔴") or (not q_comeca and ultimo_jogador == "❌"):
                return "q_venceu"
            else:
                return "minimax_venceu"
        else:
            return "empate"

# Minimax com poda alfa-beta
def minimax(jogo, turno_max, jogador, profundidade_max=3, alfa=float("-inf"), beta=float("inf")):
    if jogo.venceu() or jogo.empate() or profundidade_max == 0:
        return jogo.calcular_utilidade(jogador)

    if turno_max:
        melhor = float("-inf")
        for movimento in jogo.jogos_validos():
            resultado = jogo.jogar(movimento)
            if resultado:
                valor = minimax(resultado, False, jogador, profundidade_max - 1, alfa, beta)
                melhor = max(melhor, valor)
                alfa = max(alfa, melhor)
                if beta <= alfa:
                    break
        return melhor
    else:
        pior = float("inf")
        for movimento in jogo.jogos_validos():
            resultado = jogo.jogar(movimento)
            if resultado:
                valor = minimax(resultado, True, jogador, profundidade_max - 1, alfa, beta)
                pior = min(pior, valor)
                beta = min(beta, pior)
                if beta <= alfa:
                    break
        return pior

def melhor_jogada(jogo, profundidade=2):
    movimentos = jogo.jogos_validos()
    if not movimentos:
        return None

    if random.random() < 0.2:
        return random.choice(movimentos)

    if jogo.ultima_jogada_agente in movimentos and len(movimentos) > 1:
        movimentos.remove(jogo.ultima_jogada_agente)

    melhor_valor = float("-inf")
    melhores_movimentos = []

    for movimento in movimentos:
        resultado = jogo.jogar(movimento)
        if resultado:
            valor = minimax(resultado, False, jogo.turno(), profundidade)
            if valor > melhor_valor:
                melhor_valor = valor
                melhores_movimentos = [movimento]
            elif valor == melhor_valor:
                melhores_movimentos.append(movimento)

    return random.choice(melhores_movimentos) if melhores_movimentos else movimentos[0]

# Funções de interface
def mostrar_ajuda():
    print("\n=== REGRAS DO QUIXO ===")
    print("1. O tabuleiro é 5x5")
    print("2. Objetivo: formar uma linha de 5 peças do seu símbolo (horizontal, vertical ou diagonal)")
    print("3. Regras de movimento:")
    print("   - Só pode mover peças da borda")
    print("   - Só pode mover peças vazias ou suas peças")
    print("   - A peça é removida e empurrada do lado oposto")
    print("   - Sua peça sempre fica do lado para onde empurrou")
    print("4. Formato da jogada: origem destino (ex: 1,1 5,1)")
    print("======================\n")

def menu_principal():
    # Menu principal do jogo
    print("\n" + "="*50)
    print("\n\n ❌🔴❌🔴❌🔴❌🔴 BEM-VINDO AO JOGO QUIXO ❌🔴❌🔴❌🔴❌🔴 \n\n")
    print("="*50)
    print("1. 👤 Jogar contra Minimax")
    print("2. 🤖 Jogar contra Q-Learning")
    print("3. 🧠 Treinar Q-Learning")
    print("4. 🔄 Q-Learning vs Minimax")
    print("5. 📊 Avaliar modelos")
    print("6. 💾 Carregar modelo Q-Learning")
    print("7. ❓ Ajuda")
    print("8. 🚪 Sair")
    print("="*50)

def jogar_contra_humano(agente_tipo="minimax", agente_q=None):
    # Permite jogar contra um dos agentes
    estado = Jogo(jogador='🔴')
    contador_turnos = 0
    max_turnos = 100

    simbolo_agente = "❌"
    nome_agente = "Minimax" if agente_tipo == "minimax" else "Q-Learning"

    print(f"\n🎯 Você (🔴) vs {nome_agente} ({simbolo_agente})")
    print("🆘 Digite 'ajuda' para ver as regras")
    print("📄 Digite 'historico' para ver as últimas jogadas")
    print("🔚 Digite 'sair' para voltar ao menu\n")

    while contador_turnos < max_turnos:
        estado.imprimir()

        if estado.venceu():
            vencedor = '🔴' if estado.jogador_atual == '❌' else '❌'
            print(f"🏆 VITÓRIA DO JOGADOR {vencedor}! 🏆")
            estado.historico.imprimir_historico()
            break
        elif estado.empate():
            print("🤝 EMPATE! 🤝")
            estado.historico.imprimir_historico()
            break

        if estado.turno() == "🔴":  # Turno do usuário
            while True:
                try:
                    entrada = input("Sua jogada (origem destino): ").strip().lower()

                    if entrada == "ajuda":
                        mostrar_ajuda()
                        continue
                    elif entrada == "historico":
                        estado.historico.imprimir_historico()
                        continue
                    elif entrada == "sair":
                        return

                    origem, destino = entrada.split()
                    novo_estado = estado.jogar((origem, destino))

                    if novo_estado:
                        estado.historico.adicionar_jogada("🔴", origem, destino)
                        estado = novo_estado
                        break
                    else:
                        print("❌ Movimento inválido! Lembre-se das regras do Quixo.")
                except (ValueError, IndexError):
                    print("⚠️ Formato inválido. Use: 'linha,coluna linha,coluna' (ex: 1,1 5,1)")
        else:  # Turno do agente
            print(f"\n🤖 {nome_agente} pensando...\n")

            if agente_tipo == "minimax":
                mov = melhor_jogada(estado)
            else:  # Q-Learning
                if agente_q is None:
                    print("❌ Agente Q-Learning não carregado!")
                    return
                mov = agente_q.escolher_acao(estado, modo="teste")

            if mov:
                origem, destino = mov
                print(f"🤖 {nome_agente} move de {origem} para {destino}\n")
                estado.historico.adicionar_jogada("❌", origem, destino)
                estado.ultima_jogada_agente = mov
                estado = estado.jogar(mov)
            else:
                print(f"{nome_agente} não conseguiu encontrar um movimento válido.")
                break

        contador_turnos += 1

    if contador_turnos >= max_turnos:
        print("Jogo encerrado por número excessivo de turnos.")

def demonstrar_q_vs_minimax(agente_q, num_jogos=10):
    # Demonstra jogos entre Q-Learning e Minimax
    print(f"\n🔥 DEMONSTRAÇÃO: Q-Learning vs Minimax ({num_jogos} jogos)")
    print("="*50)

    treinador = TreinadorQLearning(agente_q)

    for i in range(num_jogos):
        print(f"\n🎮 JOGO {i+1}/{num_jogos}")
        print("-" * 30)

        # Alterna quem começa
        q_comeca = i % 2 == 0
        jogo = Jogo(jogador="🔴" if q_comeca else "❌")
        contador = 0
        max_jogadas = 30

        print(f"{'Q-Learning (🔴)' if q_comeca else 'Minimax (🔴)'} vs {'Minimax (❌)' if q_comeca else 'Q-Learning (❌)'}")

        while not jogo.venceu() and not jogo.empate() and contador < max_jogadas:
            jogador_atual = "Q-Learning" if ((q_comeca and jogo.turno() == "🔴") or (not q_comeca and jogo.turno() == "❌")) else "Minimax"

            if jogador_atual == "Q-Learning":
                acao = agente_q.escolher_acao(jogo, modo="teste")
            else:
                acao = melhor_jogada(jogo, profundidade=2)

            if acao:
                origem, destino = acao
                print(f"{jogador_atual} ({jogo.turno()}): {origem} → {destino}")
                jogo = jogo.jogar(acao)

            if jogo is None:
                break
            contador += 1

        # Resultado
        if jogo and jogo.venceu():
            ultimo_jogador = "❌" if jogo.turno() == "🔴" else "🔴"
            if (q_comeca and ultimo_jogador == "🔴") or (not q_comeca and ultimo_jogador == "❌"):
                print("🏆 Q-Learning VENCEU!")
            else:
                print("🏆 Minimax VENCEU!")
        else:
            print("🤝 EMPATE!")

        if jogo:
            jogo.imprimir()

        input("Pressione Enter para continuar...")

def treinar_interface():
    # Interface para treinamento do Q-Learning
    print("\n🧠 CONFIGURAÇÃO DO TREINAMENTO")
    print("="*40)

    try:
        episodios = int(input("Número de episódios (padrão: 5000): ") or "5000")
        oponente = input("Oponente [random/minimax] (padrão: random): ").lower() or "random"
        salvar_freq = int(input("Salvar a cada N episódios (padrão: 1000): ") or "1000")
    except ValueError:
        print("❌ Valores inválidos. Usando configurações padrão.")
        episodios = 5000
        oponente = "random"
        salvar_freq = 1000

    if oponente not in ["random", "minimax"]:
        oponente = "random"

    print(f"\n🚀 Iniciando treinamento:")
    print(f"📊 Episódios: {episodios}")
    print(f"🎯 Oponente: {oponente}")
    print(f"💾 Salvar a cada: {salvar_freq} episódios")

    confirmar = input("\nContinuar? [s/N]: ").lower()
    if confirmar != 's':
        return None

    # Cria e treina o agente
    agente = QLearningAgent(jogador="❌")
    treinador = TreinadorQLearning(agente)

    try:
        treinador.treinar(
            num_episodios=episodios,
            oponente=oponente,
            salvar_a_cada=salvar_freq
        )
        print("✅ Treinamento concluído com sucesso!")
        return agente
    except KeyboardInterrupt:
        print("\n⏹️ Treinamento interrompido pelo usuário.")
        agente.salvar_modelo("q_learning_parcial.pkl")
        print("💾 Modelo parcial salvo como 'q_learning_parcial.pkl'")
        return agente

def avaliar_modelos(agente_q):
    # Interface para avaliação de modelos
    print("\n📊 AVALIAÇÃO DE MODELOS")
    print("="*30)

    try:
        num_jogos = int(input("Número de jogos para avaliação (padrão: 100): ") or "100")
    except ValueError:
        num_jogos = 100

    treinador = TreinadorQLearning(agente_q)
    resultados = treinador.avaliar_contra_minimax(num_jogos)

    # Análise estatística simples
    taxa_vitoria_q = (resultados['q_learning'] / resultados['total']) * 100
    taxa_vitoria_minimax = (resultados['minimax'] / resultados['total']) * 100

    print(f"\n📈 ANÁLISE ESTATÍSTICA:")
    print(f"Taxa de vitória Q-Learning: {taxa_vitoria_q:.1f}%")
    print(f"Taxa de vitória Minimax: {taxa_vitoria_minimax:.1f}%")

    if taxa_vitoria_q > taxa_vitoria_minimax:
        print("🎯 Q-Learning demonstrou desempenho superior!")
    elif taxa_vitoria_minimax > taxa_vitoria_q:
        print("🎯 Minimax demonstrou desempenho superior!")
    else:
        print("🤝 Desempenho equilibrado entre os algoritmos!")

    return resultados

# Execução principal do jogo
if __name__ == "__main__":
    agente_qlearning = None

    while True:
        menu_principal()

        try:
            opcao = input("Escolha uma opção: ").strip()

            if opcao == '1':
                jogar_contra_humano("minimax")

            elif opcao == '2':
                if agente_qlearning is None:
                    print("❌ Nenhum agente Q-Learning carregado!")
                    print("💡 Carregue um modelo (opção 6) ou treine um novo (opção 3)")
                else:
                    jogar_contra_humano("qlearning", agente_qlearning)

            elif opcao == '3':
                agente_qlearning = treinar_interface()

            elif opcao == '4':
                if agente_qlearning is None:
                    print("❌ Nenhum agente Q-Learning carregado!")
                    print("💡 Carregue um modelo (opção 6) ou treine um novo (opção 3)")
                else:
                    try:
                        num = int(input("Quantos jogos demonstrar? (padrão: 5): ") or "5")
                        demonstrar_q_vs_minimax(agente_qlearning, num)
                    except ValueError:
                        demonstrar_q_vs_minimax(agente_qlearning, 5)

            elif opcao == '5':
                if agente_qlearning is None:
                    print("❌ Nenhum agente Q-Learning carregado!")
                else:
                    avaliar_modelos(agente_qlearning)

            elif opcao == '6':
                nome_arquivo = input("Nome do arquivo (padrão: q_learning_model.pkl): ").strip()
                if not nome_arquivo:
                    nome_arquivo = "q_learning_model.pkl"

                agente_qlearning = QLearningAgent()
                if agente_qlearning.carregar_modelo(nome_arquivo):
                    print(f"✅ Modelo carregado com sucesso de {nome_arquivo}")
                    print(f"📊 Estados na tabela Q: {len(agente_qlearning.q_table)}")
                    print(f"🎯 Epsilon atual: {agente_qlearning.epsilon:.4f}")
                else:
                    print(f"❌ Erro ao carregar modelo de {nome_arquivo}")
                    agente_qlearning = None

            elif opcao == '7':
                mostrar_ajuda()

            elif opcao == '8':
                print("👋 Obrigado por jogar! Até a próxima!")
                break

            else:
                print("❌ Opção inválida! Escolha um número de 1 a 8.")

        except KeyboardInterrupt:
            print("\n\n👋 Programa encerrado pelo usuário.")
            break
        except Exception as e:
            print(f"❌ Erro inesperado: {e}")
            print("🔄 Retornando ao menu principal...")

        input("\nPressione Enter para continuar...")




 ❌🔴❌🔴❌🔴❌🔴 BEM-VINDO AO JOGO QUIXO ❌🔴❌🔴❌🔴❌🔴 


1. 👤 Jogar contra Minimax
2. 🤖 Jogar contra Q-Learning
3. 🧠 Treinar Q-Learning
4. 🔄 Q-Learning vs Minimax
5. 📊 Avaliar modelos
6. 💾 Carregar modelo Q-Learning
7. ❓ Ajuda
8. 🚪 Sair
Escolha uma opção: 3

🧠 CONFIGURAÇÃO DO TREINAMENTO


👋 Programa encerrado pelo usuário.
